# Exam project code

In [1]:
# Scraping ft.dk to obtain a dataset of members of the danish parliament

import matplotlib.pyplot as plt
import numpy as np 
import pandas as pd
import seaborn as sns
import requests
import re
from bs4 import BeautifulSoup
import time
import urllib.request


from urllib3.exceptions import InsecureRequestWarning

/opt/anaconda3/lib/python3.7/site-packages/statsmodels/tools/_testing.py:19: FutureWarning: pandas.util.testing is deprecated. Use the functions in the public API at pandas.testing instead.
  import pandas.util.testing as tm


In [2]:

# Header to be used in request (with contact info) 
header = {'Name':'Bjørn Bremholm', 'Email':'bjornbremholm@gmail.com', 'Phone':'0045 31417797', 
            'Description':'Gather list of MPs for research project at KU'}
# header = {'Name':'Bjørn Bremholm', 'Email':'bjornbremholm@gmail.com'}

url = 'https://www.ft.dk/da/medlemmer/mandatfordelingen'
# requests.packages.urllib3.disable_warnings(category=InsecureRequestWarning)
# header['Info'] = 'Gather list of MP for research project'
page = requests.get(url,verify=False, headers=header)
soup = BeautifulSoup(page.text, 'html.parser')

# page
# soup.find_all('table')
# Get the links
data = soup.find_all('table')[0]
aref = data.find_all('a')
member_nr = data.find_all('div', {'class':'member-container'})

# Create a list with all relevant infomation 
Parties = []

for i in range(len(aref)):
    Parties.append([aref[i].text, aref[i].get('href'),member_nr[i].text])

# Creates a Dataframe    
df_Parties = pd.DataFrame(Parties, columns=['Party','AddressLink','Members'])
df_Parties

/opt/anaconda3/lib/python3.7/site-packages/urllib3/connectionpool.py:847: InsecureRequestWarning: Unverified HTTPS request is being made. Adding certificate verification is strongly advised. See: https://urllib3.readthedocs.io/en/latest/advanced-usage.html#ssl-warnings
  InsecureRequestWarning)


,Party,AddressLink,Members
0,Socialdemokratiet (S),/searchResults.aspx?sortedDescending=false&par...,49
1,Venstre (V),/searchResults.aspx?sortedDescending=false&par...,39
2,Dansk Folkeparti (DF),/searchResults.aspx?sortedDescending=false&par...,16
3,Socialistisk Folkeparti (SF),/searchResults.aspx?sortedDescending=false&par...,15
4,Radikale Venstre (RV),/searchResults.aspx?sortedDescending=false&par...,14
5,Enhedslisten (EL),/searchResults.aspx?sortedDescending=false&par...,13
6,Det Konservative Folkeparti (KF),/searchResults.aspx?sortedDescending=false&par...,13
7,Nye Borgerlige (NB),/searchResults.aspx?sortedDescending=false&par...,4
8,Liberal Alliance (LA),/searchResults.aspx?sortedDescending=false&par...,3
9,Alternativet (ALT),/searchResults.aspx?sortedDescending=false&par...,1


In [3]:
ID_list = []

for i in range(len(df_Parties)): # Loop for each party
    Party = df_Parties['Party'][i]

    #Get the relevant party page
    link = df_Parties['AddressLink'][i]
    url = f'http://ft.dk{link}&page=1&sortedBy=&pageSize=50'
    # time.sleep(1)
    page = requests.get(url, verify=False, headers=header)
    soup = BeautifulSoup(page.text, 'html.parser')

    # Get the names 
    tables = soup.find_all('tr',{'tabindex':0})

    for i in range(len(tables)): # Loop for each member of the given party 
        FirstName = tables[i].find_all('td')[1].text
        LastName = tables[i].find_all('a')[0].text
        Name = FirstName + ' ' + LastName
        MP = tables[i].find_all('td')[4].text
        ID_list.append([Name,Party,MP])

df_Name = pd.DataFrame(ID_list, columns=['Name','Party','MP'])

/opt/anaconda3/lib/python3.7/site-packages/urllib3/connectionpool.py:847: InsecureRequestWarning: Unverified HTTPS request is being made. Adding certificate verification is strongly advised. See: https://urllib3.readthedocs.io/en/latest/advanced-usage.html#ssl-warnings
  InsecureRequestWarning)
/opt/anaconda3/lib/python3.7/site-packages/urllib3/connectionpool.py:847: InsecureRequestWarning: Unverified HTTPS request is being made. Adding certificate verification is strongly advised. See: https://urllib3.readthedocs.io/en/latest/advanced-usage.html#ssl-warnings
  InsecureRequestWarning)
/opt/anaconda3/lib/python3.7/site-packages/urllib3/connectionpool.py:847: InsecureRequestWarning: Unverified HTTPS request is being made. Adding certificate verification is strongly advised. See: https://urllib3.readthedocs.io/en/latest/advanced-usage.html#ssl-warnings
  InsecureRequestWarning)
/opt/anaconda3/lib/python3.7/site-packages/urllib3/connectionpool.py:847: InsecureRequestWarning: Unverified HTT

In [4]:
df_Name.to_csv('NameId.csv')
df_Parties.to_csv('Parties.csv')

In [5]:
pd.set_option('display.max_rows', None)
df_Name

,Name,Party,MP
0,Ida Auken,Socialdemokratiet (S),Medlem af Folketinget
1,Trine Bramsen,Socialdemokratiet (S),Medlem af Folketinget
2,Bjørn Brandenborg,Socialdemokratiet (S),Medlem af Folketinget
3,Jeppe Bruus,Socialdemokratiet (S),Medlem af Folketinget
4,Morten Bødskov,Socialdemokratiet (S),Medlem af Folketinget
5,Lennart Damsbo-Andersen,Socialdemokratiet (S),Medlem af Folketinget
6,Kaare Dybvad Bek,Socialdemokratiet (S),Medlem af Folketinget
7,Benny Engelbrecht,Socialdemokratiet (S),Medlem af Folketinget
8,Camilla Fabricius,Socialdemokratiet (S),Medlem af Folketinget
9,Mette Frederiksen,Socialdemokratiet (S),Medlem af Folketinget


# Scraping Twitter

In [192]:
%matplotlib inline
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
from IPython.display import YouTubeVideo

#pip3 install tweepy
import tweepy
import configparser
config = configparser.ConfigParser()

# Import packages

import seaborn as sns
import requests
import os
import re
from bs4 import BeautifulSoup
import time

import json
# import tweepy
import webbrowser


In [191]:
pip install tweepy

  Using cached https://files.pythonhosted.org/packages/a3/12/b92740d845ab62ea4edf04d2f4164d82532b5a0b03836d4d4e71c6f3d379/requests_oauthlib-1.3.0-py2.py3-none-any.whl
     |████████████████████████████████| 153kB 4.3MB/s 
Note: you may need to restart the kernel to use updated packages.


In [100]:
url = 'https://filip.sdu.dk/twitter/politikere/'
page = requests.get(url)
soup = BeautifulSoup(page.text, 'html.parser')

data = soup.find_all('div', {'class':'col-md-6 show_tweet show_user'}) # Finds the list of all politicians twitter account

Twitter_list = []

for i in range(len(data)):
    info_i = data[i].find_all('div', {'class':'media-body'})
    twitter_id = info_i[0].find('h3').text.split()
    person_id = info_i[0].find('small').text.split(' \nmed ')

    Name = person_id[0]
    Tag = twitter_id[1]
    Følgere = re.search('[0-9.]+[0-9]|\d', person_id[1]).group()

    Twitter_list.append([Name,Tag,Følgere])

df_Twitter_id = pd.DataFrame(Twitter_list, columns=['Name','Twitter_id','Følgere'])
df_Twitter_id.head(5)

,Name,Twitter_id,Følgere
0,Margrethe Vestager,@vestager,295.327
1,Lars Løkke Rasmussen,@larsloekke,198.345
2,Pernille Skipper,@PSkipperEL,82.871
3,Ida Auken,@IdaAuken,73.926
4,Kristian Jensen,@Kristian_Jensen,62.946


In [134]:
# manuel løsning af handles
manuel_handles = {'Kaare Dybvad Bek':'@KaareDybvad', 'Karin Gaardsted' : '@KarinGaardsted', 'Ane Halsboe-Jørgensen':'@AneHalsboe', 
                    'Christian Rabjerg Madsen' : '@RabjergMadsen', 'Lars Aslan Rasmussen':'@lars_aslan', 'Pernille Rosenkrantz-Theil' : '@RosenkrantzT',
                    'Kasper Roug':'@KasperRoug', 'Mads  Fuglede':'@madsfuglede', 'Peter Juel-Jensen':'@PeterJuelJensen',
                    'Stén Knuth':'@Sten_Knuth',  'Lars Christian Lilleholt':'@larsclilleholt', 'Kristian Pihl Lorentzen':'@kplorentzen',
                    'Torsten Schack Pedersen':'@Torstenschack', 'Lise Bech':'@LiseBech', 'Jens Henrik Thulesen Dahl':'@JThulesen', 
                    'Mette Hjermind Dencker':'@dfmehd_mette', 'Kirsten Normann Andersen':'@KirstenNormann', 'Karina Lorentzen Dehnhardt':'@MF_K_Lorentzen',
                    'Charlotte Broman Mølbæk':'@charlottebroman', 'Rasmus Nordqvist':'@rasmusnordqvist', 'Trine Torp':'@TrineTorp', 
                    'Sofie Carsten Nielsen':'@sofiecn', 'Rasmus Helveg Petersen':'@rasmushelveg', 'Victoria Velasquez':'@VictoriaV_EL', 
                    'Katarina Ammitzbøll':'@Ammitzboell_K', 'Brigitte Klintskov Jerkel':'@JerkelK', 'Aki-Matilda Høegh-Dam': '@AkiMati_Siumut',
                    'Sjúrður Skaale': '@SjurSkaale', 'Uffe Elbæk':'@uffeelbaek'}


In [182]:
for key in manuel_handles:
    index = df_info[df_info['Name'] == key].index
    df_info.iloc[index[0],3] = manuel_handles[key]


In [188]:

def choose_color(some_name, color):
    index = df_info[df_info['Name'] == some_name].index[0]
    df_info.iloc[index[0],?] = color


175

In [183]:
df_Twitter_missing = df_info.loc[df_info['Twitter_id'] == 'None']
df_info_list = df_info.loc[df_info['Twitter_id'] != 'None']

In [186]:
df_info.to_csv('twitter_handles.csv')

In [111]:
single_names = []
for i in range(len(df_Twitter_missing)):
    single_names.append(re.split(r"\s+", df_Twitter_missing.iloc[i,0]))

# df_Name.iloc[62,0]
single_names

[['Lennart', 'Damsbo-Andersen'],
 ['Kaare', 'Dybvad', 'Bek'],
 ['Mette', 'Frederiksen'],
 ['Karin', 'Gaardsted'],
 ['Ane', 'Halsboe-Jørgensen'],
 ['Daniel', 'Toft', 'Jakobsen'],
 ['Henrik', 'Dam', 'Kristensen'],
 ['Christian', 'Rabjerg', 'Madsen'],
 ['Henrik', 'Møller'],
 ['Lars', 'Aslan', 'Rasmussen'],
 ['Pernille', 'Rosenkrantz-Theil'],
 ['Kasper', 'Roug'],
 ['Claus', 'Hjort', 'Frederiksen'],
 ['Mads', 'Fuglede'],
 ['Preben', 'Bang', 'Henriksen'],
 ['Peter', 'Juel-Jensen'],
 ['Stén', 'Knuth'],
 ['Lars', 'Christian', 'Lilleholt'],
 ['Kristian', 'Pihl', 'Lorentzen'],
 ['Christoffer', 'Aagaard', 'Melson'],
 ['Kenneth', 'Mikkelsen'],
 ['Torsten', 'Schack', 'Pedersen'],
 ['Alex', 'Ahrendtsen'],
 ['Lise', 'Bech'],
 ['René', 'Christensen'],
 ['Jens', 'Henrik', 'Thulesen', 'Dahl'],
 ['Mette', 'Hjermind', 'Dencker'],
 ['Kirsten', 'Normann', 'Andersen'],
 ['Karina', 'Lorentzen', 'Dehnhardt'],
 ['Charlotte', 'Broman', 'Mølbæk'],
 ['Rasmus', 'Nordqvist'],
 ['Ina', 'Strøjer-Schmidt'],
 ['Trine', 

In [120]:
ft_names = single_names
handles = str(df_Twitter_id['Name'])
# handles = df_Twitter_id['Name']
handles

fornavne = [item[0] for item in ft_names]

# mellemnavne

for x in fornavne:
    if re.search(x,handles):
        # print(re.search(x,handles))
        print(x)

# results = []
# for j in range(len(df_Twitter_id)):
#     for i in range(len(single_names)):
#         ft_names = single_names[i]
#         for x in ft_names:
#             if re.search(x,handles[j]):
#                 # print('match')
#                 # print(re.search(x,handles))
#                 results.append(handles[j])
#             # else:
#             #     print('no match')

# results


Kaare
Mette
Karin
Ane
Daniel
Henrik
Christian
Henrik
Lars
Pernille
Kasper
Claus
Mads
Preben
Peter
Stén
Lars
Kristian
Christoffer
Kenneth
Torsten
Alex
Lise
Jens
Mette
Kirsten
Karina
Charlotte
Rasmus
Trine
Sofie
Rasmus
Jette
Henning
Victoria
Katarina
Mads
Brigitte
Uffe


In [181]:
# Merge the Dataframes 
df_info = df_Name.merge(df_Twitter_id,how='left')
df_info.replace(float('NaN'),'None',inplace=True)

# Create two new dataframes - the ones with (without) twitter
df_Twitter_missing = df_info.loc[df_info['Twitter_id'] == 'None']
df_info_list = df_info.loc[df_info['Twitter_id'] != 'None']


# df_info_list
df_Twitter_missing


,Name,Party,MP,Twitter_id,Følgere
5,Lennart Damsbo-Andersen,Socialdemokratiet (S),Medlem af Folketinget,None,None
6,Kaare Dybvad Bek,Socialdemokratiet (S),Medlem af Folketinget,None,None
9,Mette Frederiksen,Socialdemokratiet (S),Medlem af Folketinget,None,None
11,Karin Gaardsted,Socialdemokratiet (S),Stedfortræder,None,None
12,Ane Halsboe-Jørgensen,Socialdemokratiet (S),Medlem af Folketinget,None,None
17,Daniel Toft Jakobsen,Socialdemokratiet (S),Medlem af Folketinget,None,None
27,Henrik Dam Kristensen,Socialdemokratiet (S),Medlem af Folketinget,None,None
34,Christian Rabjerg Madsen,Socialdemokratiet (S),Medlem af Folketinget,None,None
36,Henrik Møller,Socialdemokratiet (S),Medlem af Folketinget,None,None
40,Lars Aslan Rasmussen,Socialdemokratiet (S),Medlem af Folketinget,None,None


In [193]:
#keys:
consumer_key = 'WKJJfH4lNwJoKALXfTGNIqERT'
consumer_secret  = 'kWtQ0x1i7mz1ffS0aoYARAaYGn74jdeyUQCcwqPmy2YkcXFxc5'

callback_url = 'oob'
auth = tweepy.OAuthHandler(consumer_key, consumer_secret, callback_url)

try:
    redirect_url = auth.get_authorization_url()
except tweepy.TweepError:
    print('Error! Failed to get request token.')

webbrowser.open(redirect_url)
print(redirect_url)

user_pin_input = input('What is the pin value? ')
auth.get_access_token(user_pin_input)
api = tweepy.API(auth)

https://api.twitter.com/oauth/authorize?oauth_token=ksa5JwAAAAABSvVWAAABe08q378


In [194]:
# Function gathering the tweets from a specific user/handle

def extract_tweets(handle):

    # Input: Handle (e.g. @JeppeKofod) which is used to collect tweets
    #
    # Output: Dataframe containing tweets attaching information

    # tweets = api.user_timeline(screen_name=handle,count=200,exclude_replies=True,include_rts=False,tweet_mode="extended") 
    tweets = tweepy.Cursor(api.user_timeline,screen_name = handle, tweet_mode='extended', include_rts=False).items()
    tweets_data = []

    for tweet in tweets:
        date = tweet.created_at
        author = tweet.author.screen_name
        likes = tweet.favorite_count
        retweets = tweet.retweet_count
        source = tweet.source
        text = tweet.full_text

        tweets_data.append([date,author,text,likes,retweets,source])

    df_tweets = pd.DataFrame(tweets_data, columns=['Date','Author','Tweet','Likes','Retweets','Source'])
    return df_tweets 

In [195]:
handles = ['@larsloekke']
df_Tweets = pd.DataFrame()

for handle in handles:
    a = extract_tweets(handle)
    df_Tweets = df_Tweets.append(a,ignore_index=True)

df_Tweets.head(5)

,Date,Author,Tweet,Likes,Retweets,Source
0,2021-08-15 22:01:12,larsloekke,@FantaskMT @Anettelarner @PukDamsgaard Sludder...,6,0,Twitter Web App
1,2021-08-10 10:02:59,larsloekke,"@KarenVLoomersc2 Så det, straks jeg havde send...",10,0,Twitter Web App
2,2021-08-10 09:56:54,larsloekke,"... og der skulle have stået ""vigtigt"". Bare i...",69,4,Twitter Web App
3,2021-08-10 09:54:32,larsloekke,Vigtig indspark i debatten. Når vi år efter år...,95,7,Twitter Web App
4,2021-08-07 12:14:03,larsloekke,@ChristinaEgelun Tillykke.,4,1,Twitter Web App
5,2021-08-06 18:46:44,larsloekke,"@DrDahlberg Tak @DrDahlberg - og godt, at der ...",9,0,Twitter Web App
6,2021-08-05 13:03:45,larsloekke,@plauritsen Tak.,8,0,Twitter Web App
7,2021-08-04 20:33:00,larsloekke,@SinkbS @VilasHJensen Det tør jeg da godt svar...,1,0,Twitter Web App
8,2021-08-04 20:15:34,larsloekke,"@VilasHJensen Tillykke med den, Vilas.",8,0,Twitter Web App
9,2021-08-04 20:05:06,larsloekke,"@rulgw 1.000 tak, Rulle. Varme lidt oven på en...",22,0,Twitter Web App


In [27]:
# Trying to scrape FB
url = 'https://www.facebook.com/mettefrederiksen.dk'

response = requests.get(url)
response

soup = BeautifulSoup(response.text, 'html.parser')
# soup

# soup.find_all('div')



In [135]:
from selenium import webdriver
from webdriver_manager.chrome import ChromeDriverManager
import time

url = 'https://www.facebook.com/'
driver = webdriver.Chrome(ChromeDriverManager().install())
driver.get(url)
time.sleep(2)
driver.find_element_by_xpath('/html/body/div[3]/div[2]/div/div/div/div/div[3]/button[2]').click()

email = driver.find_element_by_xpath('//*[@id="email"]')
password = driver.find_element_by_xpath('//*[@id="pass"]')
email.send_keys('bjornbremholm@gmail.com')

time.sleep(6)

search = driver.find_element_by_xpath('/html/body/div[1]/div/div[1]/div/div[2]/div[2]/div/div/div/div/div/label/input')
search.send_keys(full_names[9])
time.sleep(3)
# search.send_keys(' \n')
# time.sleep(3)

driver.find_element_by_xpath('//*[@id="mette frederiksen"]/div/a').click()
time.sleep(3)



# fb_page = driver.find_elements_by_partial_link_text('mette')[0]
# fb_page.click()


# fb_page = driver.find_elements_by_tag_name('a')
# time.sleep(1)
# fb_page.click
# driver.close







====== WebDriver manager ======
Current google-chrome version is 92.0.4515
Get LATEST driver version for 92.0.4515
Driver [/Users/bjornbremholm/.wdm/drivers/chromedriver/mac64/92.0.4515.107/chromedriver] found in cache


In [118]:
test= driver.find_elements_by_tag_name('a')
test[0]#.get_attribute('innerHTML')

<selenium.webdriver.remote.webelement.WebElement (session="fb8a50b3de29826d2c239d58dfe2982d", element="c787cac6-3729-4866-96ee-8c01639594d4")>

In [132]:
driver

<selenium.webdriver.chrome.webdriver.WebDriver (session="fb8a50b3de29826d2c239d58dfe2982d")>

In [138]:
elle = driver.find_element_by_xpath('//*[@aria-label="Søgeresultater"]')
elle.find_elements_by_xpath('.//*[@href]')

[<selenium.webdriver.remote.webelement.WebElement (session="ebfcd36ea2eb444ec55af3791c1f1a3e", element="ee6ec3c3-9608-4a7f-b2c1-42a08b67e45f")>,
 <selenium.webdriver.remote.webelement.WebElement (session="ebfcd36ea2eb444ec55af3791c1f1a3e", element="f56e4b3c-443c-4990-9c0f-8a69a39829e3")>,
 <selenium.webdriver.remote.webelement.WebElement (session="ebfcd36ea2eb444ec55af3791c1f1a3e", element="01d682d7-0104-48bb-9b7c-4f2f05e23dbd")>,
 <selenium.webdriver.remote.webelement.WebElement (session="ebfcd36ea2eb444ec55af3791c1f1a3e", element="93c0abd4-516e-4373-8457-89a1efe67115")>,
 <selenium.webdriver.remote.webelement.WebElement (session="ebfcd36ea2eb444ec55af3791c1f1a3e", element="24ffc9c3-6a4d-4784-890d-c5f6666a9018")>,
 <selenium.webdriver.remote.webelement.WebElement (session="ebfcd36ea2eb444ec55af3791c1f1a3e", element="85ca1540-6397-45e7-8a27-5d5b62bfa6cd")>,
 <selenium.webdriver.remote.webelement.WebElement (session="ebfcd36ea2eb444ec55af3791c1f1a3e", element="c29a1d76-4ad1-4364-a620-07

In [141]:
elle = driver.find_element_by_xpath('//*[@aria-label="Søgeresultater"]')
for i in elle.find_elements_by_xpath('.//*[@href]'):
    #print(i.text)
    if i.text == full_names[9]:
        print(i.text)
        i.click()
#elle.find_element_by_tag_name('a')
#elle.get_attribute('innerHTML')

Mette Frederiksen
None


In [125]:
full_names[9]

'Mette Frederiksen'

In [52]:
full_names = []
for i in range(len(df_Name)):
    full_names.append(str(df_Name['First Name'].loc[i]) + ' ' + str(df_Name['Last Name'].loc[i]))
full_names[9]

'Mette Frederiksen'